In [1]:
import os

os.environ["CHECKPOINTS_PATH"] = "../checkpoints"

import blur_detector
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy.orm import Session
from src.api.db import engine
from src.api.services import simrooms_service, labeling_service

from experiment.settings import (
    CLASS_ID_TO_NAME,
    LABELING_REC_SAME_BACKGROUND_ID,
    RECORDINGS_PATH,
    BLUR_METRICS_PATH,
)
from src.utils import extract_frames_to_dir
from tqdm import tqdm
import tempfile
from pathlib import Path
from concurrent.futures import as_completed, ThreadPoolExecutor
import shutil

## Laplacian variance:

Laplacian Variance measures image detail by evaluating the variance in pixel intensities, with a higher value indicating greater image sharpness 

## Spatially Varying Blur Detection

See https://github.com/Utkarsh-Deshmukh/Spatially-Varying-Blur-Detection-python and the paper https://arxiv.org/pdf/1703.07478

Generates a blur map, with higher intensities being sharper and lower intensities being blurry

In [2]:
import warnings
from concurrent.futures import ProcessPoolExecutor

from scipy.stats import trim_mean

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)


def compute_robust_metrics(blur_map):
    # Flatten the blur_map to a 1D array of high frequency coefficients.
    h_freq_values = blur_map.flatten()

    # Raw 90th percentile of the raw data.
    raw_percentile90 = np.percentile(h_freq_values, 90)

    # Winsorize the data by clipping the lower 5% and upper 95% of the values.
    lower_bound = np.percentile(h_freq_values, 5)
    upper_bound = np.percentile(h_freq_values, 95)
    winsorized = np.clip(h_freq_values, lower_bound, upper_bound)
    winsorized_90 = np.percentile(winsorized, 90)

    # Calculate a 10% trimmed mean (dropping 10% of the extreme values on each tail).
    trimmed = trim_mean(h_freq_values, proportiontocut=0.1)

    # Compute the median.
    median_val = np.median(h_freq_values)

    # Compute the median absolute deviation (MAD).
    mad_val = np.median(np.abs(h_freq_values - median_val))

    return {
        "raw_percentile90": raw_percentile90,
        "winsorized_90": winsorized_90,
        "trimmed_mean": trimmed,
        "median": median_val,
        "mad": mad_val,
    }


# Define a function to process a single annotation file.
def process_frame(frame_path: Path):
    frame_idx = int(frame_path.stem)
    frame = cv2.imread(frame_path)
    # grame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # blur_map = blur_detector.detectBlur(
    #     grame_gray,
    #     downsampling_factor=1,
    #     num_scales=4,
    #     scale_start=1,
    #     show_progress=False,
    # )
    laplacian_variance = cv2.Laplacian(frame, cv2.CV_64F).var()

    # Basic variance metric.
    # blur_var = blur_map.var()

    # # Compute robust metrics from the blur_map.
    # robust_metrics = compute_robust_metrics(blur_map)

    # Build the result dictionary with all the metrics.
    result = {
        "frame_idx": frame_idx,
        "laplacian_variance": laplacian_variance,
        # "blur_var": blur_var,
        # "raw_percentile90": robust_metrics["raw_percentile90"],
        # "winsorized_90": robust_metrics["winsorized_90"],
        # "trimmed_mean": robust_metrics["trimmed_mean"],
        # "median": robust_metrics["median"],
        # "mad": robust_metrics["mad"],
    }

    return result

In [3]:
if BLUR_METRICS_PATH.exists():
    shutil.rmtree(BLUR_METRICS_PATH)
BLUR_METRICS_PATH.mkdir(parents=True, exist_ok=True)

recordings = list(RECORDINGS_PATH.glob("*.mp4"))
if len(recordings) != 16:
    raise ValueError(f"Expected 16 recordings, found {len(recordings)}")

# For every annotated class in the recording:
for recording_path in recordings:
    recording_id = recording_path.stem

    print(f"Extracting frames for {recording_id}")
    tmp_frames_dir = tempfile.TemporaryDirectory()
    tmp_frames_path = Path(tmp_frames_dir.name)
    extract_frames_to_dir(
        video_path=recording_path,
        frames_path=tmp_frames_path,
        print_output=False,
    )
    frames = sorted(list(tmp_frames_path.glob("*.jpg")), key=lambda x: int(x.stem))

    print(f"Calculating blur metrics for {recording_id}")
    result_rows = []
    with ProcessPoolExecutor() as executor:
        future_to_path = {
            executor.submit(process_frame, frame_path): frame_path
            for frame_path in frames
        }

        # As each future completes, gather its results.
        for future in tqdm(
            as_completed(future_to_path),
            total=len(future_to_path),
            desc=f"Processing frames for recording {recording_id}",
        ):
            try:
                result = future.result()
                result_rows.append(result)
            except Exception as exc:
                failed_path = future_to_path[future]
                print(f"Error processing {failed_path}: {exc}")

    # Create a DataFrame with the collected results.
    result_df = pd.DataFrame(result_rows)
    result_df.to_csv(BLUR_METRICS_PATH / f"{recording_id}.csv", index=False)

Extracting frames for 5235be94-da01-43b5-8827-92a51d32ce30
Calculating blur metrics for 5235be94-da01-43b5-8827-92a51d32ce30


Processing frames for recording 5235be94-da01-43b5-8827-92a51d32ce30: 100%|██████████| 1368/1368 [00:20<00:00, 66.67it/s]


Extracting frames for d6fd0aed-b901-4863-bad8-7910dad693e0
Calculating blur metrics for d6fd0aed-b901-4863-bad8-7910dad693e0


Processing frames for recording d6fd0aed-b901-4863-bad8-7910dad693e0: 100%|██████████| 14121/14121 [03:07<00:00, 75.25it/s]


Extracting frames for d50c5f3b-2822-4462-9880-5a8f0dd46bfb
Calculating blur metrics for d50c5f3b-2822-4462-9880-5a8f0dd46bfb


...

Processing frames for recording 32f02db7-adc0-4556-a2da-ed2ba60a58c9: 100%|██████████| 1365/1365 [00:19<00:00, 70.19it/s]


Extracting frames for 67823ccd-a1f0-4cde-b954-3b9e5fe160c1
Calculating blur metrics for 67823ccd-a1f0-4cde-b954-3b9e5fe160c1


Processing frames for recording 67823ccd-a1f0-4cde-b954-3b9e5fe160c1: 100%|██████████| 1554/1554 [00:18<00:00, 85.27it/s]


Extracting frames for 89b60530-e0e4-4f5d-9ee6-af85c8d99ff4
Calculating blur metrics for 89b60530-e0e4-4f5d-9ee6-af85c8d99ff4


Processing frames for recording 89b60530-e0e4-4f5d-9ee6-af85c8d99ff4: 100%|██████████| 1270/1270 [00:15<00:00, 82.74it/s]


In [ ]:
import math

plt.figure(figsize=(30, 7))
plt.plot(frame_indexes, laplacian_vars)
plt.title("Laplacian variance per frame")
plt.xlabel("Frame index")
plt.ylabel("Laplacian variance")
plt.show()

# plot a histogram of the laplacian variance
plt.figure(figsize=(15, 7))
counts, bin_edges, patches = plt.hist(
    laplacian_vars, bins=100
)  # Using 100 bins as in your example
plt.title("Laplacian variance histogram")
plt.xlabel("Laplacian variance")
plt.ylabel("Frequency")
plt.show()

import random

if not isinstance(laplacian_vars, np.ndarray):
    laplacian_vars = np.array(laplacian_vars)

# 1. Define the overall variance range and bin properties
min_overall_variance = 0.0
max_overall_variance = 5.0
bin_width = 1.0
num_examples_per_bin = 10

# Create bin edges (e.g., [0.0, 1.0, 2.0, ..., 10.0])
# Use arange for float steps, then ensure the max_overall_variance is included if it's a multiple of bin_width
bin_edges = np.arange(min_overall_variance, max_overall_variance + bin_width, bin_width)
# Ensure the last edge doesn't slightly exceed max_overall_variance due to float precision if it was meant to be exact
if bin_edges[-1] > max_overall_variance and np.isclose(
    bin_edges[-1], max_overall_variance
):
    bin_edges[-1] = max_overall_variance
elif (
    bin_edges[-1] < max_overall_variance
    and len(bin_edges) * bin_width - bin_width < max_overall_variance
):  # if arange stopped short
    # This case is less likely with + bin_width, but good to be aware
    pass


print(
    f"Defined bins for variance range [{min_overall_variance}, {max_overall_variance}) with width {bin_width}:"
)
# Bin definitions: [start, end)
target_bins = []
for i in range(len(bin_edges) - 1):
    bin_start = bin_edges[i]
    bin_end = bin_edges[i + 1]
    # Ensure we don't exceed the overall max, especially for the last bin
    if (
        bin_start < max_overall_variance
    ):  # only consider bins that start before the overall max
        target_bins.append({
            "start": bin_start,
            "end": min(bin_end, max_overall_variance),
        })
        print(
            f"  Bin {i}: [{target_bins[-1]['start']:.1f}, {target_bins[-1]['end']:.1f})"
        )


# 2. Collect examples for each target bin
all_selected_examples_by_bin = {}  # To store lists of {'path': Path, 'variance': float}

for i, bin_info in enumerate(target_bins):
    bin_start, bin_end = bin_info["start"], bin_info["end"]
    bin_label = f"[{bin_start:.1f}-{bin_end:.1f})"

    # Find frames that fall within this specific bin
    # Note: The upper limit of the bin is exclusive (var < bin_end)
    # For the very last bin that should include max_overall_variance, adjust if needed,
    # but typically ranges are [start, end).
    # If bin_end is exactly max_overall_variance, we want var < max_overall_variance or var <= max_overall_variance
    # Let's stick to [start, end) for consistency.

    current_bin_frames_data = []
    for frame_idx, var_val in enumerate(laplacian_vars):
        # Check if var_val is in [bin_start, bin_end)
        if bin_start <= var_val < bin_end:
            current_bin_frames_data.append({
                "path": frames[frame_idx],
                "variance": var_val,
            })

    # Select random examples from this bin
    if current_bin_frames_data:
        if len(current_bin_frames_data) > num_examples_per_bin:
            all_selected_examples_by_bin[bin_label] = random.sample(
                current_bin_frames_data, num_examples_per_bin
            )
        else:
            all_selected_examples_by_bin[bin_label] = (
                current_bin_frames_data  # Show all if fewer
            )
    else:
        all_selected_examples_by_bin[bin_label] = []  # No images in this bin
        print(f"No images found for bin {bin_label}")


# 3. Plot the selected example images
num_bins_with_examples = sum(
    1 for ex_list in all_selected_examples_by_bin.values() if ex_list
)

if num_bins_with_examples > 0:
    # Plot each bin's examples in a separate figure row or figure for clarity
    # Let's try one figure with multiple rows, one row per bin. Max 5 images per row.

    # Calculate total number of images to plot to set up the figure
    total_images_to_plot = sum(
        len(ex_list) for ex_list in all_selected_examples_by_bin.values()
    )

    if total_images_to_plot == 0:
        print("No example images to plot.")
    else:
        # Max 5 columns (for the 5 examples per bin)
        cols = num_examples_per_bin
        # Number of rows is the number of bins that have examples
        rows = num_bins_with_examples

        fig_height_per_row = 3  # inches
        fig_width_per_col = 3  # inches
        plt.figure(figsize=(cols * fig_width_per_col, rows * fig_height_per_row))

        plot_idx = 1  # Global plot index for subplots
        current_row_title_y = 0.9  # Initial Y for row titles

        bin_counter = 0
        for bin_label, examples in all_selected_examples_by_bin.items():
            if not examples:
                continue  # Skip bins with no examples

            # Add a title for this bin's row of images (optional, can get crowded)
            # For simplicity, we'll just ensure titles are on individual plots.

            for example_data in examples:
                example_image_path = example_data["path"]
                actual_variance = example_data["variance"]

                img = cv2.imread(str(example_image_path))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                ax = plt.subplot(rows, cols, plot_idx)
                plt.imshow(img)

                # Add bin label to the title of the first image in the row for context
                if (plot_idx - 1) % cols == 0:
                    ax.set_title(
                        f"Bin: {bin_label}\nVar: {actual_variance:.2f}", fontsize=9
                    )
                else:
                    ax.set_title(f"Var: {actual_variance:.2f}", fontsize=9)
                plt.axis("off")
                plot_idx += 1

            # Fill remaining columns in the row if fewer than num_examples_per_bin were shown
            while (plot_idx - 1) % cols != 0 and plot_idx <= rows * cols:
                # Add an empty subplot to maintain grid structure if a bin has < 5 examples
                ax = plt.subplot(rows, cols, plot_idx)
                ax.axis("off")  # Make it invisible
                plot_idx += 1

            bin_counter += 1

        plt.suptitle(
            f"Example Images from Laplacian Variance Bins in Range [{min_overall_variance:.1f}, {max_overall_variance:.1f})",
            fontsize=14,
        )
        plt.tight_layout(rect=[0, 0.02, 1, 0.96])  # Adjust layout
        plt.show()
else:
    print(
        f"No frames found with Laplacian variance between {min_overall_variance} and {max_overall_variance} to pick examples from."
    )

# You might also want to see the histogram again to understand where these ranges fall
plt.figure(figsize=(15, 7))
plt.hist(laplacian_vars, bins=100, label="All Variances", zorder=1)
# Highlight the selected BINS on the histogram
for i, bin_info in enumerate(target_bins):
    color = plt.cm.get_cmap("viridis", len(target_bins))(
        i
    )  # Different color for each bin span
    plt.axvspan(
        bin_info["start"],
        bin_info["end"],
        color=color,
        alpha=0.3,
        zorder=2,
        label=f"Bin [{bin_info['start']:.1f}-{bin_info['end']:.1f})"
        if i % 2 == 0
        else None,
    )  # Label every other to avoid clutter
plt.title("Laplacian Variance Histogram with Target Bins Highlighted")
plt.xlabel("Laplacian Variance")
plt.ylabel("Frequency")
# plt.legend() # Can be too cluttered with many bins, enable if needed
plt.show()